In [41]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("23tanmay/BioDistillGPT2")
model = AutoModelForCausalLM.from_pretrained("23tanmay/BioDistillGPT2")

# Input prompt
input_prompt = "I am having head ache cold cough amd fever since last 3 days and cough is getting worse day by day accompanied by breathing problem. I took some medicines but they didn't work much"

# Tokenize the input prompt
input_ids = tokenizer.encode(input_prompt, return_tensors="pt")

# Generate the response
output_ids = model.generate(
    input_ids, 
    max_length=800,
    num_return_sequences=3,
    # no_repeat_ngram_size=2,
    # temperature=0.7,
    top_p=0.95,
    top_k=50,
    do_sample=True
)

response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
response2=tokenizer.decode(output_ids[1], skip_special_tokens=True)
response3=tokenizer.decode(output_ids[2], skip_special_tokens=True)
print("Input Prompt:", input_prompt)
print("response 1: ", response)
print("response 2: ",response2)
print("response 3: ",response3)
response_string = response+response2+response3

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Input Prompt: I am having head ache cold cough amd fever since last 3 days and cough is getting worse day by day accompanied by breathing problem. I took some medicines but they didn't work much
response 1:  I am having head ache cold cough amd fever since last 3 days and cough is getting worse day by day accompanied by breathing problem. I took some medicines but they didn't work much. So I am taking the medicines and taking rest. I will be fine for next few days. Please suggest me the proper diet.
[RESPONSE]Hello rearward welcome to Chat Doctor.come have evaluated your query in details.* There are different reasons for the same as upper respiratory tract infection to this stage which may be viral or bacterial.@ Recommendations for better recovery-Gargles with salted lukewarm water added peppermint oil 3 times a day.- Plenty of liquids with balanced nutritious diet to have enough liquids and basic electrolytes.  $ The medicines should be guided by the clinical physician by a specialis

In [42]:
# import spacy
# import os

# # Load the SpaCy language model
# nlp = spacy.load("en_core_web_sm")
# def extract_pos(text, pos_tags=["NOUN", "ADJ", "ADV"]):
#     """
#     Extract specified parts of speech from the text.
#     Args:
#         text (str): The input text.
#         pos_tags (list): List of POS tags to extract (default: ["NOUN", "ADJ", "ADV"]).
#     Returns:
#         list: Extracted tokens with specified POS tags.
#     """
#     doc = nlp(text)
#     extracted = [token.text for token in doc if token.pos_ in pos_tags]
#     return extracted

# # Extract from response
# extracted_string = str(extract_pos(response_string))
# print("Extracted Terms:", extracted_string)


In [43]:
# import spacy
# import textacy.preprocessing as tp
# nlp = spacy.load("en_core_web_sm", disable=("parser",))
# def extract_nouns_verbs(text):
#     text = tp.normalize.whitespace(text)
#     text = tp.remove.punctuation(text)
    
#     doc = nlp(text)

#     unique_nouns_verbs = set()
#     for token in doc:
#         if token.pos_ in ["NOUN","ADJ","ADV"]:
#             unique_nouns_verbs.add(token.text)
    
#     return " ".join(unique_nouns_verbs)
# response_keywords=str(extract_nouns_verbs(response_string))
# print(response_keywords)

In [44]:
import pandas as pd
df=pd.read_csv("disease_description.csv")

In [45]:
df.head()

,idx,disease,Symptom,reason,TestsAndProcedures,commonMedications
0,0,Panic disorder,"['Anxiety and nervousness', 'Depression', 'Sho...",Panic disorder is an anxiety disorder characte...,"['Psychotherapy', 'Mental health counseling', ...","['Lorazepam', 'Alprazolam (Xanax)', 'Clonazepa..."
1,1,Vocal cord polyp,"['Hoarse voice', 'Sore throat', 'Difficulty sp...","beclomethasone nasal product,","['Tracheoscopy and laryngoscopy with biopsy', ...","['Esomeprazole (Nexium)', 'Beclomethasone Nasa..."
2,2,Turner syndrome,"['Groin mass', 'Leg pain', 'Hip pain', 'Suprap...",Turner syndrome or Ullrich\xe2\x80\x93Turner s...,"['Complete physical skin exam performed (ML)',...","['Somatropin', 'Sulfamethoxazole (Bactrim)', '..."
3,3,Cryptorchidism,"['Symptoms of the scrotum and testes', 'Swelli...",Cryptorchidism (derived from the Greek \xce\xb...,"['Complete physical skin exam performed (ML)',...","['Haemophilus B Conjugate Vaccine (Obsolete)',..."
4,4,Poisoning due to ethylene glycol,"['Abusing alcohol', 'Fainting', 'Hostile behav...","thiamine,","['Intravenous fluid replacement', 'Hematologic...","['Lorazepam', 'Thiamine', 'Naloxone (Suboxone)..."


In [46]:
df.dtypes

idx                    int64
disease               object
Symptom               object
reason                object
TestsAndProcedures    object
commonMedications     object
dtype: object

## Handling score for disease speciality csv

In [47]:
score_metrics = {
    'disease': 4,
    'Symptom': 2,
    'reason': 1,
    'TestsAndProcedures': 1,
    'commonMedications': 1
}

In [48]:
def calculate_relevance_score(keyword, column_value, column_name):
    if isinstance(column_value, list):
        for item in column_value:
            if keyword in item:
                return score_metrics[column_name]  # Return score metric for the column
    else:
        if keyword in str(column_value):
            return score_metrics[column_name]  # Return score metric for the column
    return 0  # Return 0 if keyword not found in the column

In [49]:
total_scores = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Initialize total score for the current row
    total_score = 0
    
    # Calculate relevance score for each keyword in each column
    for keyword in response_string:
        for column_name in df.columns:
            if column_name != 'idx':  # Exclude the 'idx' column
                column_value = row[column_name]
                score = calculate_relevance_score(keyword, column_value, column_name)
                total_score += score
    
    # Append total score for the current row
    total_scores.append(total_score)

# Append total scores to the DataFrame
df['Total_Score'] = total_scores

# Save the updated DataFrame to a CSV file
df.to_csv('updated_dataframe.csv', index=False)

In [50]:
updated_df=pd.read_csv("updated_dataframe.csv")

In [51]:
updated_df.head()

,idx,disease,Symptom,reason,TestsAndProcedures,commonMedications,Total_Score
0,0,Panic disorder,"['Anxiety and nervousness', 'Depression', 'Sho...",Panic disorder is an anxiety disorder characte...,"['Psychotherapy', 'Mental health counseling', ...","['Lorazepam', 'Alprazolam (Xanax)', 'Clonazepa...",19485
1,1,Vocal cord polyp,"['Hoarse voice', 'Sore throat', 'Difficulty sp...","beclomethasone nasal product,","['Tracheoscopy and laryngoscopy with biopsy', ...","['Esomeprazole (Nexium)', 'Beclomethasone Nasa...",17257
2,2,Turner syndrome,"['Groin mass', 'Leg pain', 'Hip pain', 'Suprap...",Turner syndrome or Ullrich\xe2\x80\x93Turner s...,"['Complete physical skin exam performed (ML)',...","['Somatropin', 'Sulfamethoxazole (Bactrim)', '...",19061
3,3,Cryptorchidism,"['Symptoms of the scrotum and testes', 'Swelli...",Cryptorchidism (derived from the Greek \xce\xb...,"['Complete physical skin exam performed (ML)',...","['Haemophilus B Conjugate Vaccine (Obsolete)',...",17364
4,4,Poisoning due to ethylene glycol,"['Abusing alcohol', 'Fainting', 'Hostile behav...","thiamine,","['Intravenous fluid replacement', 'Hematologic...","['Lorazepam', 'Thiamine', 'Naloxone (Suboxone)...",19521


In [52]:
updated_df["Total_Score"].min()

np.int64(12812)

## Doctor To Speciality Mapping

In [53]:
specialist_df = pd.read_csv("specialist_description.csv")
specialist_df.head()

,id,Speciality,Description,Subspeciality
0,1.0,Allergy and Immunology,Allergy and immunology is a subspecialty of in...,NaN
1,2.0,Anesthesiology,A doctor of anesthesiologyan anesthesiologistp...,"['adult cardiothoracic anesthesiology', 'criti..."
2,3.0,Cardiology,Cardiology is a subspecialty of internal medic...,['advanced heart failure and transplant cardio...
3,4.0,Colon and Rectal Surgery,Colon and rectal surgeons diagnose and treat d...,NaN
4,5.0,Dermatology,"A doctor of dermatologya dermatologistfinds, p...","['dermatopathology', 'micrographic surgery', '..."


In [54]:
# Initialize a dictionary to store scores for each specialist
scores = {}

# Split the response string into words and convert to lowercase
response_words = response_string.lower().split()

# Iterate over each word in the response string
for word in response_words:
    # Check if the word is in the "Speciality" column of speciality_df
    for index, row in specialist_df.iterrows():
        if word == row['Speciality'].lower() and word != "medicine" and word!="and":
            # Increment score by 1000
            scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 1000
        elif row['Speciality'].lower().endswith("ogy"):
            # Check if the specialty ends with "ogy" and the corresponding term is in the response string
            specialty_term = row['Speciality'].lower()[:-3] + "ogist"
            if specialty_term in response_string.lower():
                scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 1000

# Iterate over each description in speciality_df
for index, row in specialist_df.iterrows():
    description_words = row['Description'].lower().split()
    # Count occurrences of each word in the response string
    for word in description_words:
        if word in response_words:
            # Increase score by 5 for each hit
            scores[row['Speciality']] = scores.get(row['Speciality'], 0) + 5

# Sort specialists based on scores
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)

# Print the top 3 specialists with the highest score
print("Top 3 specialists:")
for i, (speciality, score) in enumerate(sorted_scores[:3], 1):
    print(f"{i}. {speciality}: {score} score")


Top 3 specialists:
1. Pulmonology: 461085 score
2. Allergy and Immunology: 240 score
3. Obstetrics and Gynecology: 230 score


In [55]:
# specialist_df["Speciality"].unique

<bound method Series.unique of 0                   Allergy and Immunology
1                           Anesthesiology
2                               Cardiology
3                 Colon and Rectal Surgery
4                              Dermatology
5                       Emergency Medicine
6                          Family Medicine
7                       Forensic Pathology
8                          General Surgery
9                    Genetics and Genomics
10         Hospice and Palliative Medicine
11                       Hospital Medicine
12                       Internal Medicine
13                               Neurology
14                    Neurological Surgery
15               Obstetrics and Gynecology
16                      Ophthalmic Surgery
17                     Orthopaedic Surgery
18                          Otolaryngology
19                               Pathology
20                              Pediatrics
21    Physical Medicine and Rehabilitation
22                     